In [33]:
import json
import pandas as pd
from time import sleep
import socket
socket.getaddrinfo('localhost', 8080)
import http.client

In [1]:
client_id = None #Insert your Seatgeek client ID here
client_secret = None #Insert you Seatgeek client secret here

In [19]:
def getPageResults(priceRange):
    #Data Frame to store the results
    info = pd.DataFrame()
    #Connect to the Seatgeek API
    conn = http.client.HTTPSConnection("api.seatgeek.com")
    #Request all Lakers events with lowest price with the price range given
    conn.request("GET", "/2/events?client_id=" + client_id +"&client_secret=" + client_secret + "&performers.slug=los-angeles-lakers&per_page=100&lowest_price.gte=" + str(priceRange[0]) + "&lowest_price.lt=" + str(priceRange[1]))
    #Read the response
    res = conn.getresponse()
    data = res.read()
    decodedData = data.decode("utf-8")
    results = json.loads(decodedData)
    eventList = results['events']
    #For each event, get all the relevant info that can be found
    for event in eventList:
        infoDict = {}
        for key in ["title", "datetime_local", "id"]:
            if key in event:
                infoDict[key] = event[key]
            else:
                infoDict[key] = None
        if "name" in event["venue"]:
            infoDict["venue name"] = event["venue"]["name"]
            infoDict["city"] = event["venue"]["city"]
        else:
            infoDict["venue name"] = None
            infoDict["city"] = None
        #Add the price ranges to the info
        infoDict["lowest price (range)"] = priceRange
        infoDict["lowest price lower bound"] = priceRange[0]
        infoDict["lowest price upper bound"] = priceRange[1]
        #Add the info to the data frame
        info = pd.concat([info, pd.DataFrame.from_dict(infoDict, orient = 'index').T])
    return info

In [28]:
#Set up the data frame of events with the correct columns
allEvents = pd.DataFrame(columns = ["title","datetime_local","id","venue name","city"])
#There shouldn't be any events in this category, but add them just to check
allEvents = pd.concat([allEvents, getPageResults([0, 50])])
sleep(15)
#Iterate through each range of $10 from 50 to 550, finding all events in that category
for i in range(50, 550, 10):
    newResults = getPageResults([i, i+10])
    sleep(15)
    #For whatever reason, the requests include all events with lower prices than the input range, so to avoid redundant results, filter out existing entries
    newResults = newResults[~newResults["id"].isin(allEvents["id"])]
    allEvents = pd.concat([allEvents, newResults])
#There shouldn't be any events in this category, but add them just to check
newResults = getPageResults([560, 10000])
sleep(15)
newResults = newResults[~newResults["id"].isin(allEvents["id"])]
allEvents = pd.concat([allEvents, newResults])

In [29]:
allEvents

,title,datetime_local,id,venue name,city,lowest price (range),lowest price lower bound,lowest price upper bound
0,Atlanta Hawks at Los Angeles Lakers,2026-01-13T19:30:00,17670611,Crypto.com Arena,Los Angeles,"[50, 60]",50,60
0,New Orleans Pelicans at Los Angeles Lakers,2026-03-03T19:30:00,17670634,Crypto.com Arena,Los Angeles,"[50, 60]",50,60
0,Los Angeles Lakers at Utah Jazz,2025-12-18T19:00:00,17524222,Delta Center,Salt Lake City,"[60, 70]",60,70
0,Los Angeles Lakers at New Orleans Pelicans,2026-01-06T19:00:00,17632249,Smoothie King Center,New Orleans,"[60, 70]",60,70
0,NBA Cup - Quarterfinals: San Antonio Spurs at ...,2025-12-10T19:00:00,17672772,Crypto.com Arena,Los Angeles,"[80, 90]",80,90
0,Milwaukee Bucks at Los Angeles Lakers,2026-01-09T19:30:00,17670608,Crypto.com Arena,Los Angeles,"[80, 90]",80,90
0,Charlotte Hornets at Los Angeles Lakers,2026-01-15T19:30:00,17670612,Crypto.com Arena,Los Angeles,"[90, 100]",90,100
0,Los Angeles Lakers at Denver Nuggets,2026-01-20T20:00:00,17670389,Ball Arena,Denver,"[90, 100]",90,100
0,Philadelphia 76ers at Los Angeles Lakers,2026-02-05T19:00:00,17670616,Crypto.com Arena,Los Angeles,"[90, 100]",90,100
0,Washington Wizards at Los Angeles Lakers,2026-03-30T10:00:00,17670649,Crypto.com Arena,Los Angeles,"[90, 100]",90,100


In [32]:
allEvents.to_csv("seatgeek_events.csv", index=False)